In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# roc curve and auc score
from sklearn.datasets import make_classification

In [3]:
from sklearn.model_selection import train_test_split
X,y=make_classification(n_samples=2000,n_classes=2,weights=[1,1],random_state=1)

In [4]:
X.shape

(2000, 20)

In [10]:
y

array([0, 0, 0, ..., 1, 1, 0])

In [11]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [12]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# random Forest

In [13]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
ytrain_predict=rf.predict_proba(X_train)
print("RF train roc-auc:{}".format(roc_auc_score(y_train,ytrain_predict[:,1])))
ytest_predict=rf.predict_proba(X_test)
print("Rf test roc-auc:{}".format(roc_auc_score(y_test,ytest_predict[:,1])))

RF train roc-auc:1.0
Rf test roc-auc:0.9825277777777778


# Logistic Regression

In [17]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(X_train,y_train)
ytrain_predict=lr.predict_proba(X_train)
print("lr of roc-auc:{}".format(roc_auc_score(y_train,ytrain_predict[:,1])))
ytest_predict=lr.predict_proba(X_test)
print("lr test of roc-auc:{}".format(roc_auc_score(y_test,ytest_predict[:,1])))

lr of roc-auc:0.9863568922694498
lr test of roc-auc:0.9885777777777777


In [18]:
#adaboost
from sklearn.ensemble import AdaBoostClassifier
ada_classifier=AdaBoostClassifier()
ada_classifier.fit(X_train, y_train)
ytrain_pred = ada_classifier.predict_proba(X_train)
print('Adaboost train roc-auc: {}'.format(roc_auc_score(y_train, ytrain_pred[:,1])))
ytest_pred = ada_classifier.predict_proba(X_test)
print('Adaboost test roc-auc: {}'.format(roc_auc_score(y_test, ytest_pred[:,1])))

Adaboost train roc-auc: 0.9975081174960356
Adaboost test roc-auc: 0.9826111111111111


In [19]:
#knn classifier
from sklearn.neighbors import KNeighborsClassifier
knn_classifier=KNeighborsClassifier()
knn_classifier.fit(X_train, y_train)
ytrain_pred = knn_classifier.predict_proba(X_train)
print('Adaboost train roc-auc: {}'.format(roc_auc_score(y_train, ytrain_pred[:,1])))
ytest_pred = knn_classifier.predict_proba(X_test)
print('Adaboost test roc-auc: {}'.format(roc_auc_score(y_test, ytest_pred[:,1])))

Adaboost train roc-auc: 0.981670071491109
Adaboost test roc-auc: 0.9426111111111111


In [20]:
#now we will be selecting best threshold value
pred=[]
for model in [rf,lr,ada_classifier,knn_classifier]:
    pred.append(pd.Series(model.predict_proba(X_test)[:,1]))
final_prediction=pd.concat(pred,axis=1).mean(axis=1)
print("ensemble test roc-auc:{}".format(roc_auc_score(y_test,final_prediction)))

ensemble test roc-auc:0.9849222222222223


In [21]:
pd.concat(pred,axis=1)

,0,1,2,3
0,0.98,0.991861,0.559186,1.0
1,0.00,0.000008,0.463282,0.0
2,0.96,0.966929,0.538202,0.8
3,0.92,0.761539,0.509875,0.8
4,0.51,0.779443,0.490344,0.4
...,...,...,...,...
595,0.01,0.024239,0.461121,0.0
596,0.02,0.000003,0.441377,0.0
597,0.98,0.984385,0.532403,1.0
598,0.00,0.001147,0.441720,0.2


In [22]:
final_prediction

0      0.882762
1      0.115823
2      0.816283
3      0.747853
4      0.544947
         ...   
595    0.123840
596    0.115345
597    0.874197
598    0.160717
599    0.834857
Length: 600, dtype: float64

In [23]:
#calculate the roc curve
fpr,tpr,thresholds=roc_curve(y_test,final_prediction)
thresholds #but which threshold value should we slect

array([1.91172166, 0.91172166, 0.90406694, 0.90327475, 0.80498136,
       0.80412833, 0.79311406, 0.7930558 , 0.78675879, 0.78571156,
       0.76614847, 0.76537124, 0.74525211, 0.74387362, 0.71249037,
       0.71143711, 0.6692442 , 0.66743537, 0.60207293, 0.60114346,
       0.58995356, 0.58986186, 0.58577749, 0.57925354, 0.55808969,
       0.54639283, 0.54338469, 0.53627948, 0.45990225, 0.45616892,
       0.45450602, 0.43544916, 0.4321034 , 0.38920009, 0.38379719,
       0.35433098, 0.35086612, 0.24154122, 0.24146893, 0.2321341 ,
       0.23140421, 0.19916474, 0.19848417, 0.12053605, 0.1203351 ,
       0.10548439])

In [24]:
from sklearn.metrics import accuracy_score
accuracy_ls = []
for thres in thresholds:
    y_pred = np.where(final_prediction>thres,1,0)
    accuracy_ls.append(accuracy_score(y_test, y_pred, normalize=True))
    
accuracy_ls = pd.concat([pd.Series(thresholds), pd.Series(accuracy_ls)],
                        axis=1)
accuracy_ls.columns = ['thresholds', 'accuracy']
accuracy_ls.sort_values(by='accuracy', ascending=False, inplace=True)
accuracy_ls.head()

,thresholds,accuracy
29,0.456169,0.961667
31,0.435449,0.960000
30,0.454506,0.960000
28,0.459902,0.960000
32,0.432103,0.958333


In [25]:
def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()